# Plugin Manager

> Plugin discovery, loading, and lifecycle management system

In [ ]:
#| default_exp core.manager

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import importlib.metadata
import importlib.util
import inspect
import logging
from pathlib import Path
import sys
from typing import Dict, List, Optional, Type, Any, Tuple, Generator

from cjm_plugin_system.core.interface import PluginInterface
from cjm_plugin_system.core.metadata import PluginMeta

In [ ]:
#| export
# Optional: Import error handling library if available
try:
    from cjm_error_handling.core.base import ErrorContext, ErrorSeverity
    from cjm_error_handling.core.errors import PluginError, ValidationError
    _has_error_handling = True
except ImportError:
    _has_error_handling = False

## PluginManager

The `PluginManager` class handles the complete lifecycle of plugins:
- Discovery via entry points or direct module loading
- Loading and initialization
- Configuration management
- Execution with optional streaming support
- Enable/disable/reload functionality
- Cleanup and unloading

## Error Handling Integration

When the `cjm-error-handling` library is installed, `PluginManager` uses structured errors for better error tracking:

- **`PluginError`**: Raised for plugin lifecycle failures (discovery, loading, initialization, execution, unloading)
- **`ValidationError`**: Raised for invalid plugin configuration

These structured errors provide:
- User-friendly messages for display
- Debug information for developers
- Rich context (plugin_name, operation, package_name, etc.)
- Error chaining with original exception cause

**Without the library**: Falls back to returning `False` or raising standard exceptions (`ValueError`)

**Usage**:
```python
try:
    manager.load_plugin(plugin_meta, config)
except PluginError as e:
    print(f"Plugin error: {e.get_user_message()}")
    print(f"Debug: {e.get_debug_message()}")
    print(f"Plugin: {e.plugin_id}")
except ValidationError as e:
    print(f"Config error: {e.get_user_message()}")
    if e.validation_errors:
        print(f"Validation details: {e.validation_errors}")
```

In [ ]:
#| export
class PluginManager:
    """Manages plugin discovery, loading, and lifecycle."""

    def __init__(
        self,
        plugin_interface:Type[PluginInterface]=PluginInterface, # Base class/interface plugins must implement
        entry_point_group:Optional[str]=None # Optional override for entry point group name
    ):
        """Initialize the plugin manager."""
        self.plugin_interface = plugin_interface

        # Use entry_point_group from plugin interface if not explicitly provided
        if entry_point_group is None:
            self.entry_point_group = plugin_interface.entry_point_group
        else:
            self.entry_point_group = entry_point_group

        self.entry_points = []
        self.discovered = []
        self.plugins: Dict[str, PluginMeta] = {}
        self._loaded_modules: Dict[str, Any] = {}
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        # Get plugin entry points
        self.get_entry_points()

    def get_entry_points(self) -> importlib.metadata.EntryPoints: # Entry points for the configured group
        """Get plugin entry points from installed packages."""
        self.entry_points = []
        try:
            # For Python 3.10+
            entry_points = importlib.metadata.entry_points()

            # Get entry points for our group
            self.entry_points = entry_points.select(group=self.entry_point_group)
        except Exception as e:
            self.logger.error(f"Error accessing entry points: {e}")
            return self.entry_points

    def discover_plugins(self) -> List[PluginMeta]: # List of discovered plugin metadata objects
        """Discover all installed plugins via entry points."""
        self.discovered = []

        for ep in self.entry_points:
            try:
                # Get the entry point name (plugin name)
                plugin_name = ep.name

                # Get package metadata
                # In Python 3.11+, ep.dist gives us the distribution
                if hasattr(ep, 'dist') and ep.dist:
                    # Get package name and version from the distribution
                    package_name = ep.dist.name  # This is the package name
                    version = ep.dist.version
                else:
                    # Fallback: try to get distribution info from entry point value
                    # The value is like "package.module:ClassName"
                    package_name = plugin_name  # Use plugin name as fallback
                    version = "unknown"

                    # Try to get the actual package info
                    try:
                        # Extract package name from the entry point value
                        ep_value = str(ep.value) if hasattr(ep, 'value') else str(ep)
                        package_part = ep_value.split(':')[0].split('.')[0]

                        # Try to get distribution info
                        dist = importlib.metadata.distribution(package_part)
                        package_name = dist.name
                        version = dist.version
                    except Exception:
                        pass

                meta = PluginMeta(
                    name=plugin_name,
                    version=version,
                    package_name=package_name
                )
                self.discovered.append(meta)
                self.logger.info(f"Discovered plugin: {meta.name} v{meta.version} from package {meta.package_name}")

            except Exception as e:
                self.logger.error(f"Error discovering plugin {ep.name}: {e}")

        return self.discovered

    def load_plugin(
        self,
        plugin_meta:PluginMeta, # Plugin metadata
        config:Optional[Dict[str, Any]]=None # Optional configuration for the plugin
    ) -> bool: # True if successfully loaded, False otherwise
        """Load and initialize a plugin."""
        try:
            # Find the entry point and load it
            plugin_eps = self.entry_points.select(name=plugin_meta.name)

            ep = list(plugin_eps)[0]
            plugin_class = ep.load()

            # Verify it implements the required interface
            if not issubclass(plugin_class, self.plugin_interface):
                error_msg = f"Plugin {plugin_meta.name} does not implement required interface"
                self.logger.error(error_msg)
                if _has_error_handling:
                    raise PluginError(
                        message=f"Plugin does not implement required interface: {plugin_meta.name}",
                        debug_info=f"Expected {self.plugin_interface.__name__}, got {plugin_class.__name__}",
                        context=ErrorContext(
                            operation="load_plugin",
                            extra={"package_name": plugin_meta.package_name}
                        ),
                        plugin_id=plugin_meta.name
                    )
                return False

            # Instantiate and initialize the plugin
            plugin_instance = plugin_class()
            plugin_instance.initialize(config)

            # Store the plugin
            plugin_meta.instance = plugin_instance
            self.plugins[plugin_meta.name] = plugin_meta

            self.logger.info(f"Loaded plugin: {plugin_meta.name}")
            return True

        except PluginError:
            # Re-raise structured errors
            raise
        except Exception as e:
            self.logger.error(f"Error loading plugin {plugin_meta.name}: {e}")
            if _has_error_handling:
                raise PluginError(
                    message=f"Failed to load plugin: {plugin_meta.name}",
                    debug_info=f"Error during plugin loading: {str(e)}",
                    context=ErrorContext(
                        operation="load_plugin",
                        extra={"package_name": plugin_meta.package_name}
                    ),
                    plugin_id=plugin_meta.name,
                    cause=e
                )
            return False

    def load_plugin_from_module(
        self,
        module_path:str, # Path to the Python module
        config:Optional[Dict[str, Any]]=None # Optional configuration for the plugin
    ) -> bool: # True if successfully loaded, False otherwise
        """Load a plugin directly from a Python module file or package."""
        try:
            # Convert to Path object
            path = Path(module_path)

            # Load the module
            if path.is_file():
                # Single file module
                spec = importlib.util.spec_from_file_location(path.stem, path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
            else:
                # Package directory
                sys.path.insert(0, str(path.parent))
                module = importlib.import_module(path.name)

            # Find plugin classes in the module
            plugin_classes = []
            for name, obj in inspect.getmembers(module):
                if (inspect.isclass(obj) and
                    issubclass(obj, self.plugin_interface) and
                    obj != self.plugin_interface):
                    plugin_classes.append(obj)

            if not plugin_classes:
                error_msg = f"No plugin classes found in {module_path}"
                self.logger.error(error_msg)
                if _has_error_handling:
                    raise PluginError(
                        message=f"No plugin classes found in module",
                        debug_info=f"Module path: {module_path}. Expected subclass of {self.plugin_interface.__name__}",
                        context=ErrorContext(
                            operation="load_plugin_from_module",
                            extra={"module_path": str(module_path)}
                        ),
                        plugin_id=str(module_path)
                    )
                return False

            # Load the first plugin class found
            plugin_class = plugin_classes[0]
            plugin_instance = plugin_class()
            plugin_instance.initialize(config)

            # Create metadata
            meta = PluginMeta(
                name=plugin_instance.name,
                version=plugin_instance.version,
                package_name=str(module_path)
            )
            meta.instance = plugin_instance

            self.plugins[meta.name] = meta
            self._loaded_modules[meta.name] = module

            self.logger.info(f"Loaded plugin from module: {meta.name}")
            return True

        except PluginError:
            # Re-raise structured errors
            raise
        except Exception as e:
            self.logger.error(f"Error loading plugin from {module_path}: {e}")
            if _has_error_handling:
                raise PluginError(
                    message=f"Failed to load plugin from module",
                    debug_info=f"Module path: {module_path}. Error: {str(e)}",
                    context=ErrorContext(
                        operation="load_plugin_from_module",
                        extra={"module_path": str(module_path)}
                    ),
                    plugin_id=str(module_path),
                    cause=e
                )
            return False

    def unload_plugin(
        self,
        plugin_name:str # Name of the plugin to unload
    ) -> bool: # True if successfully unloaded, False otherwise
        """Unload a plugin and call its cleanup method."""
        if plugin_name not in self.plugins:
            error_msg = f"Plugin {plugin_name} not found"
            self.logger.error(error_msg)
            if _has_error_handling:
                raise PluginError(
                    message=f"Cannot unload plugin: not found",
                    debug_info=f"Plugin {plugin_name} is not loaded",
                    context=ErrorContext(operation="unload_plugin"),
                    plugin_id=plugin_name
                )
            return False

        try:
            plugin_meta = self.plugins[plugin_name]
            if plugin_meta.instance:
                plugin_meta.instance.cleanup()

            del self.plugins[plugin_name]

            # Remove from loaded modules if it was loaded that way
            if plugin_name in self._loaded_modules:
                del self._loaded_modules[plugin_name]

            self.logger.info(f"Unloaded plugin: {plugin_name}")
            return True

        except Exception as e:
            self.logger.error(f"Error unloading plugin {plugin_name}: {e}")
            if _has_error_handling:
                raise PluginError(
                    message=f"Failed to unload plugin: {plugin_name}",
                    debug_info=f"Error during cleanup: {str(e)}",
                    context=ErrorContext(operation="unload_plugin"),
                    plugin_id=plugin_name,
                    cause=e
                )
            return False

    def get_plugin(
        self,
        plugin_name:str # Name of the plugin to retrieve
    ) -> Optional[PluginInterface]: # Plugin instance if found, None otherwise
        """Get a loaded plugin instance by name."""
        if plugin_name in self.plugins:
            return self.plugins[plugin_name].instance
        return None

    def list_plugins(self) -> List[PluginMeta]: # List of metadata for all loaded plugins
        """List all loaded plugins."""
        return list(self.plugins.values())

    def execute_plugin(
        self,
        plugin_name:str, # Name of the plugin to execute
        *args, # Arguments to pass to the plugin
        **kwargs # Keyword arguments to pass to the plugin
    ) -> Any: # Result of the plugin execution
        """Execute a plugin's main functionality."""
        plugin = self.get_plugin(plugin_name)
        if not plugin:
            if _has_error_handling:
                raise PluginError(
                    message=f"Cannot execute plugin: not found or not loaded",
                    debug_info=f"Plugin {plugin_name} must be loaded before execution",
                    context=ErrorContext(operation="execute_plugin"),
                    plugin_id=plugin_name
                )
            raise ValueError(f"Plugin {plugin_name} not found or not loaded")

        if not self.plugins[plugin_name].enabled:
            if _has_error_handling:
                raise PluginError(
                    message=f"Cannot execute plugin: disabled",
                    debug_info=f"Plugin {plugin_name} is currently disabled",
                    context=ErrorContext(operation="execute_plugin"),
                    plugin_id=plugin_name
                )
            raise ValueError(f"Plugin {plugin_name} is disabled")

        try:
            return plugin.execute(*args, **kwargs)
        except Exception as e:
            if _has_error_handling:
                raise PluginError(
                    message=f"Plugin execution failed: {plugin_name}",
                    debug_info=f"Error during plugin execution: {str(e)}",
                    context=ErrorContext(operation="execute_plugin"),
                    plugin_id=plugin_name,
                    cause=e
                )
            raise

    def enable_plugin(
        self,
        plugin_name:str # Name of the plugin to enable
    ) -> bool: # True if plugin was enabled, False if not found
        """Enable a plugin."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = True
            return True
        return False

    def disable_plugin(
        self,
        plugin_name:str # Name of the plugin to disable
    ) -> bool: # True if plugin was disabled, False if not found
        """Disable a plugin without unloading it."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = False
            return True
        return False

This is a generic plugin manager that works with any `PluginInterface` subclass. It provides:
- Automatic discovery via entry points
- Manual loading from module files
- Configuration management and validation
- Plugin enable/disable/reload
- Streaming support detection

The manager automatically uses the `entry_point_group` defined in the plugin interface. If not provided explicitly, it reads from `plugin_interface.entry_point_group`.

## Configuration Management

Methods for managing plugin configuration.

In [ ]:
#| export
def get_plugin_config_schema(
    self,
    plugin_name:str # Name of the plugin
) -> Optional[Dict[str, Any]]: # Configuration schema or None if plugin not found
    """Get the configuration schema for a plugin."""
    # Find the entry point
    plugin_eps = self.entry_points.select(name=plugin_name)
    if len(plugin_eps) > 0:
        ep = list(plugin_eps)[0]
        plugin_class = ep.load()
        return plugin_class.get_config_schema()
    else:
        return None

def get_plugin_config(
    self,
    plugin_name:str # Name of the plugin
) -> Optional[Dict[str, Any]]: # Current configuration or None if plugin not found
    """Get the current configuration of a plugin."""
    plugin = self.get_plugin(plugin_name)
    if plugin:
        return plugin.get_current_config()
    return None

def update_plugin_config(
    self,
    plugin_name:str, # Name of the plugin
    config:Dict[str, Any], # New configuration
    merge:bool=True # Whether to merge with existing config or replace entirely
) -> bool: # True if successful, False otherwise
    """Update a plugin's configuration and reinitialize it."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        error_msg = f"Plugin {plugin_name} not found"
        self.logger.error(error_msg)
        if _has_error_handling:
            raise PluginError(
                message=f"Cannot update config: plugin not found",
                debug_info=f"Plugin {plugin_name} is not loaded",
                context=ErrorContext(operation="update_plugin_config"),
                plugin_id=plugin_name
            )
        return False

    try:
        # Get current config if merging
        if merge:
            current_config = plugin.get_current_config()
            config = {**current_config, **config}

        # Validate the new configuration
        is_valid, error = plugin.validate_config(config)
        if not is_valid:
            error_msg = f"Invalid configuration for {plugin_name}: {error}"
            self.logger.error(error_msg)
            if _has_error_handling:
                raise ValidationError(
                    message=f"Invalid plugin configuration",
                    debug_info=f"Validation failed for plugin {plugin_name}",
                    context=ErrorContext(
                        operation="update_plugin_config",
                        extra={"plugin_name": plugin_name}
                    ),
                    validation_errors={"config": error}
                )
            return False

        # Clean up existing resources
        plugin.cleanup()

        # Reinitialize with new config
        plugin.initialize(config)

        self.logger.info(f"Updated configuration for plugin: {plugin_name}")
        return True

    except (ValidationError, PluginError):
        # Re-raise structured errors
        raise
    except Exception as e:
        error_msg = f"Error updating plugin {plugin_name} configuration: {e}"
        self.logger.error(error_msg)
        if _has_error_handling:
            raise PluginError(
                message=f"Failed to update plugin configuration",
                debug_info=f"Error reinitializing plugin {plugin_name}: {str(e)}",
                context=ErrorContext(
                    operation="update_plugin_config",
                    extra={"plugin_name": plugin_name}
                ),
                plugin_id=plugin_name,
                cause=e
            )
        return False

def validate_plugin_config(
    self,
    plugin_name:str, # Name of the plugin
    config:Dict[str, Any] # Configuration to validate
) -> Tuple[bool, Optional[str]]: # (is_valid, error_message)
    """Validate a configuration dictionary for a plugin without applying it."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        return False, f"Plugin {plugin_name} not found"

    return plugin.validate_config(config)

def get_all_plugin_schemas(
    self
) -> Dict[str, Dict[str, Any]]: # Dictionary mapping plugin names to their schemas
    """Get configuration schemas for all loaded plugins."""
    schemas = {}
    for plugin_name in self.plugins:
        plugin = self.get_plugin(plugin_name)
        if plugin:
            schemas[plugin_name] = plugin.get_config_schema()
    return schemas

def reload_plugin(
    self,
    plugin_name:str, # Name of the plugin to reload
    config:Optional[Dict[str, Any]]=None # Optional new configuration
) -> bool: # True if successful, False otherwise
    """Reload a plugin with optional new configuration."""
    if plugin_name not in self.plugins:
        error_msg = f"Plugin {plugin_name} not found"
        self.logger.error(error_msg)
        if _has_error_handling:
            raise PluginError(
                message=f"Cannot reload plugin: not found",
                debug_info=f"Plugin {plugin_name} is not loaded",
                context=ErrorContext(operation="reload_plugin"),
                plugin_id=plugin_name
            )
        return False

    try:
        # Get the plugin metadata
        plugin_meta = self.plugins[plugin_name]

        # Get current config if not provided
        if config is None and plugin_meta.instance:
            config = plugin_meta.instance.get_current_config()

        # Unload the plugin
        if plugin_meta.instance:
            plugin_meta.instance.cleanup()

        # Reload based on how it was originally loaded
        if plugin_name in self._loaded_modules:
            # Was loaded from a module file
            module_path = plugin_meta.package_name
            del self.plugins[plugin_name]
            del self._loaded_modules[plugin_name]
            return self.load_plugin_from_module(module_path, config)
        else:
            # Was loaded from entry points
            plugin_meta.instance = None
            return self.load_plugin(plugin_meta, config)

    except (PluginError, ValidationError):
        # Re-raise structured errors
        raise
    except Exception as e:
        error_msg = f"Error reloading plugin {plugin_name}: {e}"
        self.logger.error(error_msg)
        if _has_error_handling:
            raise PluginError(
                message=f"Failed to reload plugin",
                debug_info=f"Error during plugin reload: {str(e)}",
                context=ErrorContext(
                    operation="reload_plugin",
                    extra={"plugin_name": plugin_name}
                ),
                plugin_id=plugin_name,
                cause=e
            )
        return False

# Add to the PluginManager class
PluginManager.get_plugin_config_schema = get_plugin_config_schema
PluginManager.get_plugin_config = get_plugin_config
PluginManager.update_plugin_config = update_plugin_config
PluginManager.validate_plugin_config = validate_plugin_config
PluginManager.get_all_plugin_schemas = get_all_plugin_schemas
PluginManager.reload_plugin = reload_plugin

## Streaming Support

Methods for managing plugins with streaming capabilities.

In [ ]:
#| export
def execute_plugin_stream(
    self,
    plugin_name:str, # Name of the plugin to execute
    *args, # Arguments to pass to the plugin
    **kwargs # Keyword arguments to pass to the plugin
) -> Generator[Any, None, Any]: # Generator yielding partial results, returns final result
    """Execute a plugin with streaming support if available."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        if _has_error_handling:
            raise PluginError(
                message=f"Cannot execute plugin: not found or not loaded",
                debug_info=f"Plugin {plugin_name} must be loaded before execution",
                context=ErrorContext(operation="execute_plugin_stream"),
                plugin_id=plugin_name
            )
        raise ValueError(f"Plugin {plugin_name} not found or not loaded")

    if not self.plugins[plugin_name].enabled:
        if _has_error_handling:
            raise PluginError(
                message=f"Cannot execute plugin: disabled",
                debug_info=f"Plugin {plugin_name} is currently disabled",
                context=ErrorContext(operation="execute_plugin_stream"),
                plugin_id=plugin_name
            )
        raise ValueError(f"Plugin {plugin_name} is disabled")

    # Check if plugin supports streaming
    if hasattr(plugin, 'supports_streaming') and plugin.supports_streaming():
        self.logger.info(f"Using streaming mode for plugin {plugin_name}")
        try:
            return plugin.execute_stream(*args, **kwargs)
        except Exception as e:
            if _has_error_handling:
                raise PluginError(
                    message=f"Plugin streaming execution failed: {plugin_name}",
                    debug_info=f"Error during plugin streaming: {str(e)}",
                    context=ErrorContext(operation="execute_plugin_stream"),
                    plugin_id=plugin_name,
                    cause=e
                )
            raise
    else:
        self.logger.info(f"Plugin {plugin_name} doesn't support streaming, using regular execution")
        # Fall back to regular execution wrapped in a generator
        def fallback_generator():
            try:
                result = plugin.execute(*args, **kwargs)
                yield result
                return result
            except Exception as e:
                if _has_error_handling:
                    raise PluginError(
                        message=f"Plugin execution failed: {plugin_name}",
                        debug_info=f"Error during plugin execution: {str(e)}",
                        context=ErrorContext(operation="execute_plugin_stream"),
                        plugin_id=plugin_name,
                        cause=e
                    )
                raise
        return fallback_generator()

def check_streaming_support(
    self,
    plugin_name:str # Name of the plugin to check
) -> bool: # True if plugin supports streaming
    """Check if a plugin supports streaming execution."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        return False

    # Check if plugin has the supports_streaming method and it returns True
    if hasattr(plugin, 'supports_streaming'):
        return plugin.supports_streaming()

    # Fallback: check if execute_stream is implemented
    if hasattr(plugin, 'execute_stream'):
        # Check if it's overridden from base class
        plugin_class = type(plugin)
        if hasattr(plugin_class, 'execute_stream'):
            # Try to check if it's different from PluginInterface's default
            return True

    return False

def get_streaming_plugins(
    self
) -> List[str]: # List of plugin names that support streaming
    """Get a list of all loaded plugins that support streaming."""
    streaming_plugins = []
    for plugin_name in self.plugins:
        if self.check_streaming_support(plugin_name):
            streaming_plugins.append(plugin_name)
    return streaming_plugins

# Add the methods to the PluginManager class
PluginManager.execute_plugin_stream = execute_plugin_stream
PluginManager.check_streaming_support = check_streaming_support
PluginManager.get_streaming_plugins = get_streaming_plugins

### Example: Using the PluginManager

In [ ]:
import logging
from cjm_plugin_system.core.interface import PluginInterface

# Create a simple test plugin
class TestPlugin(PluginInterface):
    def __init__(self):
        self.config = {}

    entry_point_group = 'your.plugins'
        
    @property
    def name(self) -> str:
        return "test_plugin"
    
    @property
    def version(self) -> str:
        return "1.0.0"
    
    @staticmethod
    def get_config_schema():
        return {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["fast", "slow"],
                    "default": "fast"
                }
            },
            "required": ["mode"]
        }
    
    def get_current_config(self):
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(self, config=None):
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        print(f"Initialized {self.name} with config: {self.config}")
    
    def execute(self, data):
        return f"Processed '{data}' in {self.config['mode']} mode"
    
    def is_available(self):
        return True
    
    def cleanup(self):
        print(f"Cleaning up {self.name}")

In [ ]:
# Initialize plugin manager
logging.basicConfig(level=logging.INFO)
manager = PluginManager()

print(f"Entry point group: {manager.entry_point_group}")
print(f"Plugin interface: {manager.plugin_interface.__name__}")

Entry point group: None
Plugin interface: PluginInterface


In [ ]:
# Since we don't have actual installed plugins, let's save TestPlugin to a file
# and load it from the file
import tempfile
import os

# Create a temporary plugin file
plugin_code = '''
from cjm_plugin_system.core.interface import PluginInterface
from typing import Dict, Any, Optional

class DemoPlugin(PluginInterface):
    def __init__(self):
        self.config = {}

    entry_point_group = 'your.plugins'
        
    @property
    def name(self) -> str:
        return "demo_plugin"
    
    @property
    def version(self) -> str:
        return "1.0.0"
    
    @staticmethod
    def get_config_schema():
        return {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["fast", "slow"],
                    "default": "fast"
                }
            },
            "required": ["mode"]
        }
    
    def get_current_config(self):
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(self, config=None):
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
    
    def execute(self, data):
        return f"Demo: Processed '{data}' in {self.config['mode']} mode"
    
    def is_available(self):
        return True
'''

# Write to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(plugin_code)
    temp_plugin_path = f.name

print(f"Created temporary plugin at: {temp_plugin_path}")

Created temporary plugin at: /tmp/tmpy7v24njc.py


In [ ]:
# Load the plugin from file
success = manager.load_plugin_from_module(temp_plugin_path, config={"mode": "slow"})
print(f"\nPlugin loaded: {success}")

# List loaded plugins
plugins = manager.list_plugins()
print(f"\nLoaded plugins: {len(plugins)}")
for p in plugins:
    print(f"  - {p.name} v{p.version} (enabled: {p.enabled})")

INFO:__main__.PluginManager:Loaded plugin from module: demo_plugin



Plugin loaded: True

Loaded plugins: 1
  - demo_plugin v1.0.0 (enabled: True)


In [ ]:
# Get and execute plugin
plugin = manager.get_plugin("demo_plugin")
if plugin:
    print(f"\nPlugin name: {plugin.name}")
    print(f"Plugin version: {plugin.version}")
    print(f"Is available: {plugin.is_available()}")
    
    # Execute
    result = manager.execute_plugin("demo_plugin", "test data")
    print(f"\nExecution result: {result}")


Plugin name: demo_plugin
Plugin version: 1.0.0
Is available: True

Execution result: Demo: Processed 'test data' in slow mode


In [ ]:
#| eval: false
# Test configuration management
print("\nConfiguration Management:")

# Get current config
current_config = manager.get_plugin_config("demo_plugin")
print(f"Current config: {current_config}")

# Get schema
import json
schema = manager.get_plugin_config_schema("demo_plugin")
print(f"\nConfig schema:")
print(json.dumps(schema, indent=2))

# Validate config
is_valid, error = manager.validate_plugin_config("demo_plugin", {"mode": "fast"})
print(f"\nValid config test: {is_valid}")

is_valid, error = manager.validate_plugin_config("demo_plugin", {"mode": "invalid"})
print(f"Invalid config test: {is_valid}, error: {error}")


Configuration Management:
Current config: {'mode': 'slow'}

Config schema:
null

Valid config test: True
Invalid config test: False, error: 'invalid' is not one of ['fast', 'slow']

Failed validating 'enum' in schema['properties']['mode']:
    {'type': 'string', 'enum': ['fast', 'slow'], 'default': 'fast'}

On instance['mode']:
    'invalid'


In [ ]:
# Update config
print("\nUpdating configuration:")
success = manager.update_plugin_config("demo_plugin", {"mode": "fast"})
print(f"Update successful: {success}")

# Execute with new config
result = manager.execute_plugin("demo_plugin", "new data")
print(f"Result after update: {result}")

INFO:__main__.PluginManager:Updated configuration for plugin: demo_plugin



Updating configuration:
Update successful: True
Result after update: Demo: Processed 'new data' in fast mode


In [ ]:
# Test enable/disable
print("\nEnable/Disable Test:")
manager.disable_plugin("demo_plugin")
print(f"Plugin disabled: {not manager.plugins['demo_plugin'].enabled}")

try:
    manager.execute_plugin("demo_plugin", "data")
except (ValueError, Exception) as e:
    # Handle both ValueError (without error handling) and PluginError (with error handling)
    error_msg = e.get_user_message() if hasattr(e, 'get_user_message') else str(e)
    print(f"Expected error when disabled: {error_msg}")

manager.enable_plugin("demo_plugin")
print(f"Plugin enabled: {manager.plugins['demo_plugin'].enabled}")
result = manager.execute_plugin("demo_plugin", "data")
print(f"Execution after re-enable: {result}")


Enable/Disable Test:
Plugin disabled: True
Expected error when disabled: Cannot execute plugin: disabled
Plugin enabled: True
Execution after re-enable: Demo: Processed 'data' in fast mode


In [ ]:
# Test streaming support
print("\nStreaming Support:")
supports_streaming = manager.check_streaming_support("demo_plugin")
print(f"Plugin supports streaming: {supports_streaming}")

streaming_plugins = manager.get_streaming_plugins()
print(f"Streaming plugins: {streaming_plugins}")

# Try streaming execution (will fall back to regular execution)
print("\nStreaming execution:")
for chunk in manager.execute_plugin_stream("demo_plugin", "stream data"):
    print(f"  Chunk: {chunk}")

INFO:__main__.PluginManager:Plugin demo_plugin doesn't support streaming, using regular execution



Streaming Support:
Plugin supports streaming: False
Streaming plugins: []

Streaming execution:
  Chunk: Demo: Processed 'stream data' in fast mode


In [ ]:
# Cleanup
print("\nUnloading plugin:")
success = manager.unload_plugin("demo_plugin")
print(f"Unload successful: {success}")
print(f"Remaining plugins: {len(manager.list_plugins())}")

# Clean up temporary file
os.unlink(temp_plugin_path)
print(f"\nCleaned up temporary plugin file")

INFO:__main__.PluginManager:Unloaded plugin: demo_plugin



Unloading plugin:
Unload successful: True
Remaining plugins: 0

Cleaned up temporary plugin file


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()